### Classifica e Organiza

Script para analisar todos os arquivos em uma pasta, classificar e colocar na pasta correta. (direita, esquerda, frente).

Útil para curadoria automatizada.

In [27]:
import os
import cv2
import numpy as np
import shutil
from tensorflow.keras.models import load_model

### Pré-processamento da imagem

In [28]:

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (160, 120))
    image = image / 255.0
    return image


### Configurações

In [29]:

# modelo
model = load_model('modelo_direcao.keras')

# imagens
input_directory = 'fotos'

# destino
output_directories = {
    0: 'fotos/direita',
    1: 'fotos/esquerda',
    2: 'fotos/frente'
}

for dir in output_directories.values():
    os.makedirs(dir, exist_ok=True)


### Função para não sobrescrever arquivos com o mesmo nome

In [30]:

def get_unique_filename(directory, filename):
    base, extension = os.path.splitext(filename)
    counter = 1
    new_filename = filename
    while os.path.exists(os.path.join(directory, new_filename)):
        new_filename = f"{base}_{counter}{extension}"
        counter += 1
    return new_filename


### Classifica e organiza

In [31]:

for image_name in os.listdir(input_directory):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(input_directory, image_name)
        image = preprocess_image(image_path)
        image = np.expand_dims(image, axis=0)

        prediction = model.predict(image)
        class_index = np.argmax(prediction)

        output_directory = output_directories[class_index]
        unique_filename = get_unique_filename(output_directory, image_name)
        output_path = os.path.join(output_directory, unique_filename)

        shutil.move(image_path, output_path)
        print(f"Imagem {image_name} movida para {output_directory} como {unique_filename}")
